# Analysis of Census Data

Analysis of the 2013-2017 American Community Survey 5-year Public Use Microdata Sample files of population and housing unit records. 

Introduction and indicator section for website 

Counts, not percentages 


### Configuration

Load the software packages needed. 

In [1]:
import numpy as np 
import pandas as pd
import community_profiles.datasets as cp_data

### Load Census Data

In [2]:
pumas = cp_data.PUMAs.get().sort_values('puma_id')
house = cp_data.houses.get()
persons = cp_data.persons.get() 

### Population

In [3]:
group = persons.groupby('PUMA')
population = group['PWGTP'].sum().reset_index() 
population.head()

,PUMA,PWGTP
0,3201,155888
1,3202,147797
2,3203,132229
3,3204,169887
4,3205,166737


### Population by age and sex

Male = 1
Female = 2 

0-9, 10-19, 20-29, ect. 

 MAYBE BETTER WAY TO DO THIS? 

In [4]:
bins =  np.array([0,10,20,30,40,50, 60, 70, 80,100])  
group_sex = persons.groupby([pd.cut(persons.AGEP, bins, right = False), 'SEX', 'PUMA'])
population_age_sex = group_sex['PWGTP'].sum().reset_index()

In [5]:
population_age_sex.head()

,AGEP,SEX,PUMA,PWGTP
0,"[0, 10)",1,3201,9029
1,"[0, 10)",1,3202,11543
2,"[0, 10)",1,3203,9691
3,"[0, 10)",1,3204,10407
4,"[0, 10)",1,3205,14113


#### Age under 18 

In [6]:
under18 = persons.loc[persons['AGEP'] < 18 ]
groups = under18.groupby([ 'SEX', 'PUMA'])
under18 = groups['PWGTP'].sum().reset_index() 
under18.head()

,SEX,PUMA,PWGTP
0,1,3201,15325
1,1,3202,19991
2,1,3203,18194
3,1,3204,19733
4,1,3205,24285


#### Age 65 and older

In [7]:
plus65 = persons.loc[persons['AGEP'] >= 65 ]
groups1 = plus65.groupby(['SEX', 'PUMA'])
plus65 = groups1['PWGTP'].sum().reset_index() 
plus65.head()

,SEX,PUMA,PWGTP
0,1,3201,13127
1,1,3202,7269
2,1,3203,5231
3,1,3204,8302
4,1,3205,5832


### Area and Density 

area is not right, need to change crs 

denisty is not right because area is wrong 

In [8]:
area = pumas['geometry'].area 
density = population['PWGTP'] / area 

### Foreign-born 

Not Foreign-born = 1 ,
Foreign-born = 2

In [9]:
foreign = persons.loc[persons['NATIVITY']==2]
group = foreign.groupby(['PUMA'])
foreign_born = group['PWGTP'].sum().reset_index()
foreign_born.head()

,PUMA,PWGTP
0,3201,31323
1,3202,43061
2,3203,14937
3,3204,23213
4,3205,15828


### Race and Hispanic origin

#### Race

1 .White alone

2 .Black or African American alone

3 .American Indian alone

4 .Alaska Native alone

5 .American Indian and Alaska Native tribes specified; or .American Indian or Alaska Native, not specified and no other .races

6 .Asian alone

7 .Native Hawaiian and Other Pacific Islander alone

8 .Some Other Race alone

9 .Two or More Races


In [14]:
group = persons.groupby(['RAC1P', 'PUMA'])
race = group['PWGTP'].sum().reset_index()

race = race.pivot(index='PUMA', 
                  columns='RAC1P', 
                  values = 'PWGTP')
race.head()

RAC1P,1,2,3,5,6,7,8,9
PUMA,,,,,,,,
3201,119318.0,15781.0,140.0,217.0,12846.0,136.0,4107.0,3343.0
3202,66924.0,41824.0,402.0,20.0,19974.0,16.0,14392.0,4245.0
3203,71814.0,37711.0,442.0,133.0,5411.0,178.0,12485.0,4055.0
3204,17593.0,124186.0,56.0,529.0,11389.0,25.0,11316.0,4793.0
3205,82457.0,37374.0,826.0,408.0,4832.0,125.0,34216.0,6499.0


sum other

In [15]:
other = race.iloc[:, [2,3,5,6,7]].sum(axis =1)
race['other'] = other

rename columns 

In [16]:
race = (race
        .rename(columns = {1: 'white', 2 : 'black', 6 : 'asian'})
        .drop(columns = [3,5,7,8,9])
       )
race.head()

RAC1P,white,black,asian,other
PUMA,,,,
3201,119318.0,15781.0,12846.0,7943.0
3202,66924.0,41824.0,19974.0,19075.0
3203,71814.0,37711.0,5411.0,17293.0
3204,17593.0,124186.0,11389.0,16719.0
3205,82457.0,37374.0,4832.0,42074.0



#### Hispanic 

1. Not Spanish/Hispanic/Latino

    All other # Hispanic

In [17]:
hisp =  persons.loc[persons['HISP'] != 1] 
group = hisp.groupby(['HISP', 'PUMA'])
hisp = group['PWGTP'].sum().reset_index()
hisp = hisp.pivot(index='PUMA', 
                  columns='HISP', 
                  values = 'PWGTP')
hisp = hisp.iloc[:, :].sum(axis =1)

In [18]:
race['hispanic'] = hisp

#### Race and Hispanic combined in one dataframe

In [19]:
race.head()

RAC1P,white,black,asian,other,hispanic
PUMA,,,,,
3201,119318.0,15781.0,12846.0,7943.0,11574.0
3202,66924.0,41824.0,19974.0,19075.0,29218.0
3203,71814.0,37711.0,5411.0,17293.0,29541.0
3204,17593.0,124186.0,11389.0,16719.0,25103.0
3205,82457.0,37374.0,4832.0,42074.0,81932.0


### Educational attainment ( Age 25 + with a least a bachelor's)  

SCHL

21 .Bachelor's degree

22 .Master's degree

23 .Professional degree beyond a bachelor's degree

24 .Doctorate degree


In [20]:
bach_plus =  persons.loc[(persons['SCHL'] >= 21) & (persons['AGEP'] >= 25)] 
groups = bach_plus.groupby(['PUMA'])
edu = groups['PWGTP'].sum().dropna().reset_index() 
edu.head()

,PUMA,PWGTP
0,3201,28843
1,3202,18246
2,3203,12469
3,3204,16200
4,3205,15282


### Limited English 

1 .Very well

2 .Well

3 .Not well

4 .Not at all

Limited( 3 + 4 ) 

In [21]:
lim_eng = persons.loc[persons['ENG'] >= 3]
group = lim_eng.groupby(['PUMA'])
lim_eng = group['PWGTP'].sum().reset_index()
lim_eng.head()

,PUMA,PWGTP
0,3201,11031
1,3202,19224
2,3203,7466
3,3204,11054
4,3205,16182


### Unemployed

3. unemployed

In [22]:
unem = persons.loc[persons['ESR'] == 3]
group = unem.groupby(['PUMA'])
unemploy = group['PWGTP'].sum().reset_index()
unemploy.head()

,PUMA,PWGTP
0,3201,4948
1,3202,9058
2,3203,7462
3,3204,11815
4,3205,10219


### Commuting

#### Commute time

In [23]:
group = persons.groupby(['PUMA'])
mean_commute = group['JWMNP'].mean().reset_index()
mean_commute.head()

,PUMA,JWMNP
0,3201,32.858111
1,3202,34.834478
2,3203,35.192876
3,3204,35.850242
4,3205,32.114884


#### Commute type

01 .Car, truck, or van

02 .Bus or trolley bus

03 .Streetcar or trolley car (carro publico in Puerto Rico)

04 .Subway or elevated

05 .Railroad

06 .Ferryboat

07 .Taxicab

08 .Motorcycle

09 .Bicycle

10 .Walked

11 .Worked at home

12 .Other method


In [24]:
group = persons.groupby(['JWTR', 'PUMA'])
commute_t = group['PWGTP'].sum().reset_index()
commute_t = commute_t.pivot(index='PUMA', 
                            columns='JWTR',  
                            values = 'PWGTP')
commute_t.head()

JWTR,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0
PUMA,,,,,,,,,,,,
3201,60838.0,3816.0,NaN,930.0,2721.0,NaN,52.0,NaN,31.0,1426.0,1263.0,373.0
3202,46852.0,6326.0,35.0,1470.0,1986.0,NaN,91.0,NaN,85.0,2567.0,1167.0,821.0
3203,39752.0,7044.0,243.0,2691.0,864.0,NaN,83.0,17.0,190.0,1297.0,1175.0,482.0
3204,39972.0,14307.0,85.0,3354.0,1492.0,NaN,44.0,178.0,39.0,2047.0,1557.0,1064.0
3205,33907.0,8694.0,30.0,4302.0,837.0,15.0,182.0,90.0,1105.0,3158.0,1802.0,753.0


Public 

In [25]:
public = commute_t.iloc[:, [1,2,3,4,5]].sum(axis =1)
commute_t['public'] = public

Private

In [26]:
private = commute_t.iloc[:, [0,6,7]].sum(axis =1)
commute_t['private'] = private

#### Renamed and grouped commute type dataframe

In [27]:
commute_t = (commute_t
        .rename(columns = {9: 'bike', 10 : 'walk', 11 : 'work at home', 12: 'other method'})
        .drop(columns = [1,2,3,4,5,6,7,8])
       )
commute_t.head()

JWTR,bike,walk,work at home,other method,public,private
PUMA,,,,,,
3201,31.0,1426.0,1263.0,373.0,7467.0,60890.0
3202,85.0,2567.0,1167.0,821.0,9817.0,46943.0
3203,190.0,1297.0,1175.0,482.0,10842.0,39852.0
3204,39.0,2047.0,1557.0,1064.0,19238.0,40194.0
3205,1105.0,3158.0,1802.0,753.0,13878.0,34179.0


### Number of houses 

In [29]:
group = house.groupby('PUMA')
house_count = group['WGTP'].sum().reset_index() 
house_count.head()

,PUMA,WGTP
0,3201,62364
1,3202,52807
2,3203,52704
3,3204,67267
4,3205,65464


### Household Income 

Median 

In [30]:
# says use ADJINC to adjust HINCP to constant $ 
# Don't use and get this result IDK 
group = house.groupby(['PUMA'])
med_income = (group['HINCP']).median().reset_index()
med_income.head()

,PUMA,HINCP
0,3201,56500.0
1,3202,40800.0
2,3203,41000.0
3,3204,37000.0
4,3205,29050.0


### Rent Burden (35% or more of income goes to rent)

not sure if this is right 


In [31]:
house['rent_income'] = (house['GRNTP'] * 12 / house['HINCP']) *100 
above35 = house.where(house['rent_income'] >= 35) 
group = above35.groupby(['PUMA'])
rent_burden = group['WGTP'].sum().reset_index()
rent_burden.head()
# rent_burden_percent = (rent_burden['WGTP'] / house_count['WGTP']) * 100
# rent_burden_percent

,PUMA,WGTP
0,3201.0,8927.0
1,3202.0,11262.0
2,3203.0,9878.0
3,3204.0,14165.0
4,3205.0,16031.0
